# Finger People experimental playground

In [1]:
import numpy as np
import cv2
import os
from helpers import imshow


## Input Sources
### Video file

In [30]:
cap = cv2.VideoCapture('samples/all_moves_1_360.mp4') 

# all_moves_1 sizes: 360, 180, 90, 46

### Camera

In [3]:
cap = cv2.VideoCapture(0)

# reduce frame size to speed it up
w = 180
cap.set(cv2.CAP_PROP_FRAME_WIDTH, w) 
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, w * 3/4) 


True

## Farneback
#### Dense optical flow

In [38]:
cap = cv2.VideoCapture('samples/all_moves_1_180.mp4') 

# Parameters for farneback optical flow
fb_params = dict( pyr_scale = 0.5, 
                  levels = 3, 
                  winsize = 5, 
                  iterations = 3, 
                  poly_n = 5,
                  poly_sigma = 1.2, 
                  flags = 0 )

# Take the first frame
_ret, init_frame = cap.read()
prvs = cv2.cvtColor(init_frame,cv2.COLOR_BGR2GRAY)
frame1 = cv2.flip(init_frame, 1)

# Create color image
hsv = np.zeros_like(init_frame)
hsv[...,1] = 255

while(cap.isOpened()):
    
    # Get the next frame
    _ret, frame = cap.read()
    frame = cv2.flip(frame, 1)  
    if not _ret:
        break

    # Convert the frame to gray scale
    next = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    # Calculate the dense optical flow
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, **fb_params)

    # Obtain the flow magnitude and direction angle
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])

    # Update the color image
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame',np.hstack((frame, bgr)))

    # Exit on ESC
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break

    prvs = next
        
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


-1